### Setup

In [1]:
import os
import json
import pandas as pd
from rich.panel import Panel
from rich import print as rprint

# Import util functions
from helper import set_openai_key,\
                    test_openai_api,\
                    create_openai_client,\
                    print_pretty,\
                    function_to_schema,\
                    count_tiktoken_length,\
                    safe_completion

# Import tools
from helper import generate_tree_string,\
                    get_lines_from_file,\
                    search_code, \
                    get_object_definition,\
                    generate_patches

print("Import successful!")

Import successful!


In [2]:
# Define paths
comp_dir = "konwinski-prize"
comp_kaggle_evaluation_dir = os.path.join(comp_dir, "kaggle_evaluation")
comp_kprize_setup_dir = os.path.join(comp_dir, "kprize_setup")

comp_data_zip_path = os.path.join(comp_dir, "data.a_zip")
comp_data_dir = os.path.join(comp_dir, "data")
comp_data_parquet_path = os.path.join(comp_data_dir, "data.parquet")
comp_conda_packages_dir = os.path.join(comp_data_dir, "conda_packages")
comp_pip_packages_dir = os.path.join(comp_data_dir, "pip_packages")
comp_repo_configs_dir = os.path.join(comp_data_dir, "repo_configs")
comp_repos_dir = os.path.join(comp_data_dir, "repos")

In [3]:
idx = 4 # 0,1,2,3

kprize_df = pd.read_parquet(comp_data_parquet_path)
print(kprize_df.shape)

row = kprize_df.iloc[idx]
problem_statement = row["problem_statement"]
instance_id = row["instance_id"]
repo_path = os.path.join(comp_repos_dir, f'repo__{instance_id}')

print(repo_path)

(6, 10)
konwinski-prize/data/repos/repo__astropy__astropy-16830


In [4]:
print(count_tiktoken_length(problem_statement))

print(problem_statement[:1000],"\n...")

598
KeyError: 'version_1_3_or_later' when parsing certain VOTables
### Description

When parsing VOTables (for instance) VOTables with empty integer literals in some places (e.g., MIN value="" in a VALUES element), astropy crashes with

```
KeyError: 'version_1_3_or_later'
```

### Expected behavior

Well, you *could* argue that at least some such VOTables should be rejected (a NULL value in a MIN, indeed, does not make sense) with a sensible error message; with the patch proposed in the accompanying PR, astropy emits a warning.  I notice in passing that the reproducer table passes stilts votlint.  I also note in passing that against the warning, a value="" is accepted in a PARAM (as it should, as this is the way to express NULLs).

My fix in the accompanying PR just stops the crashing, leading to a workable table.

### How to Reproduce

Try

```python
from astropy import table
table.Table.read("with-empty-min.vot", format="votable")
```

with the table from https://docs.g-vo. 
...


In [5]:
set_openai_key()

test_openai_api()

client = create_openai_client()

API key set successfully.
Hello! How can I assist you today?


### Run step by step

In [6]:
model_name = "gpt-4" # "gpt-3.5-turbo" "gpt-4-turbo" "gpt-4o-mini" "gpt-4o"

system_prompt_template = """
### Context
You are a software automation agent responsible for diagnosing and resolving issues in a repository.  
Your task is to configure and utilize the provided tools effectively.  
If the available tools are insufficient, analyze the gap, propose suitable alternatives, and justify their necessity.  
Call the function generate_patches only if you precisely identify which files need to be edited and the exact content modifications required to solve the problem.  
Never ask for permission or confirmation before executing an action.
  
### Metadata
- Repository root directory: {repo_path}
"""

system_prompt = system_prompt_template.format(repo_path=repo_path)

messages = []

In [7]:
tools = [generate_tree_string, generate_tree_string, search_code, get_object_definition, generate_patches]
tool_schemas = [function_to_schema(tool) for tool in tools]
tools_map = {tool.__name__: tool for tool in tools}

**Note:** As of now, OpenAI's GPT models have a 1024-character limit for tool function descriptions, and there isn't a GPT model that allows for a larger description length.

#### Step1

In [8]:
# Call API
prompt = "Generate a patch to resolve the following repository issue:\n{problem_statement}"
prompt = prompt.format(problem_statement=problem_statement)

messages.append({"role":"user","content":prompt})

completion = safe_completion(client,
                             model_name=model_name,
                             messages=messages,
                             tool_schemas=tool_schemas,
                             retries=2
                            )

print_pretty(completion.dict())

{
    "choices": [
        {
            "finish_reason": "tool_calls",
            "index": 0,
            "logprobs": null,
            "message": {
                "audio": null,
                "content": null,
                "function_call": null,
                "refusal": null,
                "role": "assistant",
                "tool_calls": [
                    {
                        "function": {
                            "arguments": "{\n\"answer\": \"KeyError: 'version_1_3_or_later' when parsing certain VOTables\\n\\n### Description\\n\\nWhen parsing VOTables (for instance) VOTables with empty integer literals in some places (e.g., MIN value=\\\"\\\" in a VALUES element), astropy crashes with\\n\\n```\\nKeyError: 'version_1_3_or_later'\\n```\\n\\nThis is a simple fix for the problem at hand, doing the key check in analogy to the other key checks of this sort in the few places where an index rather than the get method was used on the config object.\\n\\nOne *might* w

In [10]:
# Execute tools
tool_calls = completion.choices[0].message.tool_calls

if tool_calls:
    for tool_call in completion.choices[0].message.tool_calls:

        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        function = tools_map[function_name]
        result = function(**function_args)
        
        if function_name == "generate_patches":
            rprint(Panel(function_args["answer"], title="Answer", expand=False, border_style="green"))
    
        # Append assistant's tool call response
        messages.append({
            "role": "assistant",
            "content": None,
            "tool_calls": [
                {
                    "id": tool_call.id,
                    "type": "function",
                    "function": {
                        "name": function_name,
                        "arguments": json.dumps(function_args)
                    }
                }
            ]
        })
    
        # Append tool response correctly
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": json.dumps(result)
        })
else:
    rprint(Panel(completion.choices[0].message.content, title="Response", expand=False, border_style="blue"))

╭──────────────────────────────────────────────────── Answer ─────────────────────────────────────────────────────╮
│ KeyError: 'version_1_3_or_later' when parsing certain VOTables                                                  │
│                                                                                                                 │
│ ### Description                                                                                                 │
│                                                                                                                 │
│ When parsing VOTables (for instance) VOTables with empty integer literals in some places (e.g., MIN value="" in │
│ a VALUES element), astropy crashes with                                                                         │
│                                                                                                                 │
│ ```                                                                                                             │
│ KeyError: 'version_1_3_or_later'                                                                                │
│ ```                                                                                                             │
│                                                                                                                 │
│ This is a simple fix for the problem at hand, doing the key check in analogy to the other key checks of this    │
│ sort in the few places where an index rather than the get method was used on the config object.                 │
│                                                                                                                 │
│ One *might* want to dig deeper, though; I am not exactly sure why the version_1_3_or_later key is missing on    │
│ the example table from the bug report. But I suspect (well: hope:-) that is not critical for the bug fix.       │
│                                                                                                                 │
│ Fixes #16825.                                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

#### Step2

In [11]:
# Call API
completion = safe_completion(client,
                             model_name=model_name,
                             messages=messages,
                             tool_schemas=tool_schemas,
                             retries=2
                            )

print_pretty(completion.dict())

# Execute tools
tool_calls = completion.choices[0].message.tool_calls

if tool_calls:
    for tool_call in completion.choices[0].message.tool_calls:
    
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        function = tools_map[function_name]
        result = function(**function_args)
    
        if function_name == "generate_patches":
            rprint(Panel(function_args["answer"], title="Answer", expand=False, border_style="green"))
    
        # Append assistant's tool call response
        messages.append({
            "role": "assistant",
            "content": None,
            "tool_calls": [
                {
                    "id": tool_call.id,
                    "type": "function",
                    "function": {
                        "name": function_name,
                        "arguments": json.dumps(function_args)
                    }
                }
            ]
        })
    
        # Append tool response correctly
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": json.dumps(result)
        })
else:
    rprint(Panel(completion.choices[0].message.content, title="Response", expand=False, border_style="blue"))

{
    "choices": [
        {
            "finish_reason": "stop",
            "index": 0,
            "logprobs": null,
            "message": {
                "audio": null,
                "content": "Sure! Here's a generated patch to resolve the issue:\n\n```patch\nIndex: astropy/io/votable/tree.py\n===================================================================\n--- astropy/io/votable/tree.py  (revision 16824)\n+++ astropy/io/votable/tree.py  (working copy)\n@@ -1379,7 +1379,7 @@\n     def _parse(self, iterator, config):\n         # VOTable spec. If VOTable spec version is not specified,\n         # default to using 1.3 rules.\n-        version_1_3_or_later = config['version_1_3_or_later']\n+        version_1_3_or_later = config.get('version_1_3_or_later', False)\n\n         elem = iterator.get_current_element()\n         attrs = elem.attrib\n``` \n\nPlease apply this patch to the file specified and test it with the provided scenarios. This should resolve the KeyError issue an

╭─────────────────────────────────────────────────── Response ────────────────────────────────────────────────────╮
│ Sure! Here's a generated patch to resolve the issue:                                                            │
│                                                                                                                 │
│ ```patch                                                                                                        │
│ Index: astropy/io/votable/tree.py                                                                               │
│ ===================================================================                                             │
│ --- astropy/io/votable/tree.py  (revision 16824)                                                                │
│ +++ astropy/io/votable/tree.py  (working copy)                                                                  │
│ @@ -1379,7 +1379,7 @@                                                                                           │
│      def _parse(self, iterator, config):                                                                        │
│          # VOTable spec. If VOTable spec version is not specified,                                              │
│          # default to using 1.3 rules.                                                                          │
│ -        version_1_3_or_later = config['version_1_3_or_later']                                                  │
│ +        version_1_3_or_later = config.get('version_1_3_or_later', False)                                       │
│                                                                                                                 │
│          elem = iterator.get_current_element()                                                                  │
│          attrs = elem.attrib                                                                                    │
│ ```                                                                                                             │
│                                                                                                                 │
│ Please apply this patch to the file specified and test it with the provided scenarios. This should resolve the  │
│ KeyError issue and allow for successful parsing of VOTables.                                                    │
│                                                                                                                 │
│ Please make sure to thoroughly test this fix in different scenarios to verify that it doesn't introduce any     │
│ regressions elsewhere in the code.                                                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

#### Step3

In [ ]:
# Call API
completion = safe_completion(client,
                             model_name=model_name,
                             messages=messages,
                             tool_schemas=tool_schemas,
                             retries=2
                            )

print_pretty(completion.dict())

# Execute tools
tool_calls = completion.choices[0].message.tool_calls

if tool_calls:
    for tool_call in completion.choices[0].message.tool_calls:
    
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        function = tools_map[function_name]
        result = function(**function_args)
    
        if function_name == "generate_patches":
            rprint(Panel(function_args["answer"], title="Answer", expand=False, border_style="green"))
    
        # Append assistant's tool call response
        messages.append({
            "role": "assistant",
            "content": None,
            "tool_calls": [
                {
                    "id": tool_call.id,
                    "type": "function",
                    "function": {
                        "name": function_name,
                        "arguments": json.dumps(function_args)
                    }
                }
            ]
        })
    
        # Append tool response correctly
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": json.dumps(result)
        })
else:
    rprint(Panel(completion.choices[0].message.content, title="Response", expand=False, border_style="blue"))

#### Step4

In [ ]:
# Call API
completion = safe_completion(client,
                             model_name=model_name,
                             messages=messages,
                             tool_schemas=tool_schemas,
                             retries=2
                            )

print_pretty(completion.dict())

# Execute tools
tool_calls = completion.choices[0].message.tool_calls

if tool_calls:
    for tool_call in completion.choices[0].message.tool_calls:
    
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        function = tools_map[function_name]
        result = function(**function_args)
    
        if function_name == "generate_patches":
            rprint(Panel(function_args["answer"], title="Answer", expand=False, border_style="green"))
    
        # Append assistant's tool call response
        messages.append({
            "role": "assistant",
            "content": None,
            "tool_calls": [
                {
                    "id": tool_call.id,
                    "type": "function",
                    "function": {
                        "name": function_name,
                        "arguments": json.dumps(function_args)
                    }
                }
            ]
        })
    
        # Append tool response correctly
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": json.dumps(result)
        })
else:
    rprint(Panel(completion.choices[0].message.content, title="Response", expand=False, border_style="blue"))